In [1]:
from http.server import BaseHTTPRequestHandler, HTTPServer
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
import os
from glob import glob
import json

class HTTPRequestHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == '/total_cases':
            # Create a SparkSession
            spark = SparkSession.builder \
                .appName("Total COVID Cases") \
                .getOrCreate()

            # Define the directory containing the CSV files
            directory = "/Users/meghasingh/Desktop/Spark assignment/"

            # Find all CSV files in the directory
            file_paths = glob(os.path.join(directory, "*.csv"))

            # Read each CSV file into a DataFrame
            dfs = [spark.read.csv(file_path, header=True, inferSchema=True) for file_path in file_paths]

            # Union all DataFrames
            merged_df = dfs[0]
            for df in dfs[1:]:
                merged_df = merged_df.union(df)

            # Calculate the total number of COVID cases
            total_cases = merged_df.select(sum("cases")).collect()[0][0]

            # Stop the SparkSession
            spark.stop()

            # Send response
            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            response = {'total_cases': total_cases}
            self.wfile.write(json.dumps(response).encode())

        else:
            # Send 404 response for other paths
            self.send_response(404)
            self.end_headers()
            self.wfile.write(b'404 Not Found')

def run_server(port=8005):
    server_address = ('', port)
    httpd = HTTPServer(server_address, HTTPRequestHandler)
    print(f'Starting server on port {port}...')
    httpd.serve_forever()

if __name__ == '__main__':
    run_server()

Starting server on port 8005...


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/29 12:01:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/29 12:01:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/29 12:01:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
127.0.0.1 - - [29/Mar/2024 12:01:09] "GET /total_cases HTTP/1.1" 200 -
127.0.0.1 - - [29/Mar/2024 12:01:09] "GET /favicon.ico HTTP/1.1" 404 -
24/03/29 12:02:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/29 12:02:02 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
127.0.0.1 - - [29/Mar/2024 12:02:04] "GET /total_cases HTTP/1.1" 200 -
24/03/29 12:05:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/29 12:05:58 W